In [1]:
from flask import Flask, request
import requests
import json
import os
from config import PAGE_ACCESS_TOKEN, VERIFY_TOKEN # Assuming you will define these here

In [2]:
app = Flask(__name__)

In [3]:
# Replace these with your own tokens
PAGE_ACCESS_TOKEN = PAGE_ACCESS_TOKEN
VERIFY_TOKEN = VERIFY_TOKEN

In [10]:
# --- Dynamic Question & Answer Database ---
# The bot will respond with the value (answer) that corresponds to the key (question).
# The bot will look for an exact, case-insensitive match.
# Add or remove questions and answers as needed.
qa_database = {
    "hello": "Hello! How can I assist you today?",
    "what is your purpose?": "I am a Messenger bot designed to answer a fixed set of questions.",
    "how do i get started?": "To get started, simply type one of the questions I can answer, like 'hello' or 'what are your features?'.",
    "what are your features?": "My features include answering specific questions, providing information from my database, and sending automatic responses.",
    "thank you": "You're welcome! I'm glad I could help."
}

In [5]:
# --- Root route for testing ---
@app.route("/", methods=["GET"])
def home():
    return "Messenger bot is running!"

In [6]:
# --- Webhook route for Facebook ---
@app.route("/webhook", methods=["GET", "POST"])
def webhook():
    if request.method == "GET":
        # Verification handshake
        token_sent = request.args.get("hub.verify_token")
        return verify_fb_token(token_sent)
    else:
        # Handle messages
        output = request.get_json()
        for event in output.get("entry", []):
            for messaging_event in event.get("messaging", []):
                sender_id = messaging_event["sender"]["id"]
                if messaging_event.get("message"):
                    message_text = messaging_event["message"].get("text")
                    if message_text:
                        # Convert message to lowercase for case-insensitive matching
                        message_text_lower = message_text.lower().strip()
                        
                        # Look for a response in the QA database
                        response_text = qa_database.get(message_text_lower)
                        
                        if response_text:
                            send_message(sender_id, response_text)
                        else:
                            # Default response if no match is found
                            default_response = "I'm sorry, I don't understand that question. You can try asking one of the following:\n\n"
                            # List all available questions
                            for q in qa_database.keys():
                                default_response += f"- {q}\n"
                            send_message(sender_id, default_response)
        return "Message Processed", 200

In [7]:
def verify_fb_token(token_sent):
    if token_sent == VERIFY_TOKEN:
        return request.args.get("hub.challenge")
    return "Invalid verification token"

In [8]:
def send_message(recipient_id, text):
    """Send message back to Facebook Messenger user"""
    url = "https://graph.facebook.com/v21.0/me/messages"
    params = {"access_token": PAGE_ACCESS_TOKEN}
    headers = {"Content-Type": "application/json"}
    data = {
        "recipient": {"id": recipient_id},
        "message": {"text": text}
    }
    response = requests.post(url, params=params, headers=headers, json=data)
    if response.status_code != 200:
        print(f"Error sending message: {response.text}")

In [9]:
if __name__ == "__main__":
    app.run(port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Sep/2025 20:36:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2025 20:37:36] "GET /webhook?hub.mode=subscribe&hub.challenge=147810227&hub.verify_token=kdjjskhfjskfnklafmkasa6s45asffsdnfsdf5dsfc HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2025 20:37:44] "POST /webhook HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2025 20:37:52] "POST /webhook HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2025 20:38:05] "POST /webhook HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2025 21:19:37] "POST /webhook HTTP/1.1" 200 -
127.0.0.1 - - [19/Sep/2025 21:25:25] "POST /webhook HTTP/1.1" 200 -
